**Set environment**

In [1]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



**Input**

In [2]:
ls -d ${FD_RES}/data/*/

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_Dex/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_DMSO/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX3_Dex/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX3_DMSO/
/gpfs/fs1/data/reddylab/Kuei/out/proj_com

In [3]:
ls ${FD_RES}/data/Input1/

chr10.bed  chr14.bed  chr18.bed  chr21.bed  chr4.bed  chr8.bed  target_PER1.bed
chr11.bed  chr15.bed  chr19.bed  chr22.bed  chr5.bed  chr9.bed
chr12.bed  chr16.bed  chr1.bed   chr2.bed   chr6.bed  chrX.bed
chr13.bed  chr17.bed  chr20.bed  chr3.bed   chr7.bed  chrY.bed


In [4]:
ls ${FD_RES}/data/Input1_20x/

chr10.bed  chr14.bed  chr18.bed  chr21.bed  chr4.bed  chr8.bed  target_PER1.bed
chr11.bed  chr15.bed  chr19.bed  chr22.bed  chr5.bed  chr9.bed
chr12.bed  chr16.bed  chr1.bed   chr2.bed   chr6.bed  chrX.bed
chr13.bed  chr17.bed  chr20.bed  chr3.bed   chr7.bed  chrY.bed


## Count fragments for each chromosome

**TEST**

In [5]:
### Loop chromosomes
CHROMS=($(seq 1 22) X Y)
for idx in {0..23}; do
    echo "Index: $idx; Chromosome ${CHROMS[$idx]}"
done

Index: 0; Chromosome 1
Index: 1; Chromosome 2
Index: 2; Chromosome 3
Index: 3; Chromosome 4
Index: 4; Chromosome 5
Index: 5; Chromosome 6
Index: 6; Chromosome 7
Index: 7; Chromosome 8
Index: 8; Chromosome 9
Index: 9; Chromosome 10
Index: 10; Chromosome 11
Index: 11; Chromosome 12
Index: 12; Chromosome 13
Index: 13; Chromosome 14
Index: 14; Chromosome 15
Index: 15; Chromosome 16
Index: 16; Chromosome 17
Index: 17; Chromosome 18
Index: 18; Chromosome 19
Index: 19; Chromosome 20
Index: 20; Chromosome 21
Index: 21; Chromosome 22
Index: 22; Chromosome X
Index: 23; Chromosome Y


In [15]:
### Loop folders
FD_BEDS=($(ls -d ${FD_RES}/data/{Input?,Input?_20x,TFX?_DMSO,TFX?_Dex}/))
for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    printf "SAMPLE: %-11s; FD_BED: %s\n" ${SAMPLE} ${FD_BED}
done

SAMPLE: Input1     ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1/
SAMPLE: Input1_20x ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/
SAMPLE: Input2     ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2/
SAMPLE: Input2_20x ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/
SAMPLE: Input3     ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3/
SAMPLE: Input3_20x ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/
SAMPLE: Input4     ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4/
SAMPLE: Input4_20x ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/
SAMPLE: Input5     ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5/
SAMPLE: Input5_20x ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/
SAMPLE: TFX2_Dex   ; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_com

**RUN**

In [22]:
echo ${FD_LOG}
echo ${NODE}

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log
all


In [23]:
source ../config_duke.sh

sbatch -p ${NODE} \
    --array 0-23 \
    --mem 8G \
    -o ${FD_LOG}/count_fragment_chrom.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh -v
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=${CHROM}

### set input and output
FD_BEDS=($(ls -d ${FD_RES}/data/{Input?,Input?_20x,TFX?_DMSO,TFX?_Dex}/))
FD_OUT=${FD_RES}/count_fragment
FN_BED=${TARGET}.bed

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; count fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_CNT=${FD_OUT}/${SAMPLE}
    FN_CNT=${TARGET}.bed.gz
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_CNT}/${FN_CNT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    head -n 3 ${FD_BED}/${FN_BED}
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_CNT}
    
    ### count the fragments and arrange output to a proper bed file
    cat ${FD_BED}/${FN_BED} |\
        sort    |\
        uniq -c |\
        awk '{$(NF+1)=$1;$1=""}1'   |\
        sed -e 's/^[[:space:]]*//'  |\
        sed -e 's/[[:space:]]/\t/g' |\
        sort -k2,2n |\
        gzip -c > ${FD_CNT}/${FN_CNT}
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_CNT}/${FN_CNT}
    zcat ${FD_CNT}/${FN_CNT} | head -n 3
    echo
done

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26588656


**CHECK**

In [24]:
ls ${FD_LOG}/count_fragment_chrom.17.txt

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/count_fragment_chrom.17.txt


In [25]:
cat ${FD_LOG}/count_fragment_chrom.17.txt

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect

Slurm Array Index:  17
Time Stamp:         10-13-21+22:09:05

RUN: loop through samples; count fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1//chr18.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1/chr18.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1//chr18.bed
chr18	21483	22446
chr18	21483	22446
chr18	28214	29137

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj